In [ ]:
!pip install yfinance
!pip install finta
!pip install pandas
from finta import TA
import pandas as pd
import yfinance as yf
import glob

In [ ]:
def download_stock_price_data(folder='', years=5):
  tickers = pd.read_csv('https://pkgstore.datahub.io/core/nyse-other-listings/nyse-listed_csv/data/3c88fab8ec158c3cd55145243fe5fcdf/nyse-listed_csv.csv')
  symbols = tickers['ACT Symbol'].unique()
  for s in symbols[0:20]:
    try:
      df = yf.download([s], auto_adjust=True, period=f'{str(years)}y')
      if len(df) > 100:
        df.columns = df.columns.str.lower()
        df.index = df.index.rename('datetime')
        df.to_csv(f'{folder}{s}.csv')
    except Exception as e:
      print(f'error with {s}')
      print(repr(e))
      continue
download_stock_price_data()

In [18]:
def find_entries(df:pd.DataFrame) -> pd.DataFrame:
  df['EMA200'] = TA.EMA(df, period=200)
  df['EMA50'] = TA.EMA(df, period=50)
  df['ATR'] = TA.ATR(df)
  c1 = df.close > df.EMA200
  c2 = (df.low < df.EMA50) & (df.close > df.EMA50) & (df.open > df.EMA50)
  long_entry_condition = (c1) & (c2)
  df['entry'] = df.shift(-1).open
  df['target'] = 0.0
  df['stop'] = 0.0
  df['risk:reward'] = 0.0
  df['target_pct'] = 0.0
  df['stop_pct'] = 0.0
  df['target_profit'] = 150
  df['position_size'] = 0.0
  df.loc[long_entry_condition, 'target'] = df.entry + (1.5*df.ATR)
  df.loc[long_entry_condition, 'stop'] = df.entry - df.ATR
  df.loc[long_entry_condition, 'risk:reward'] = (df.target - df.entry) / (df.entry - df.stop)
  df.loc[long_entry_condition, 'target_pct'] = (df.target - df.entry) / (df.entry)
  df.loc[long_entry_condition, 'stop_pct'] = -(df.entry - df.stop) / (df.entry)
  df.loc[long_entry_condition, 'position_size'] = df.target_profit / df.target_pct
  entries = df.loc[long_entry_condition][['entry', 'target', 'stop', 'risk:reward', 'target_pct', 'stop_pct', 'target_profit', 'position_size']].copy()
  return entries

In [20]:
filepaths = []
trade_entries_dict = {}
for file in glob.glob('*.csv'):
  filepaths.append(file)
filepaths

['A.csv',
 'AA.csv',
 'AAC.csv',
 'AAN.csv',
 'AAP.csv',
 'AAT.csv',
 'AB.csv',
 'ABB.csv',
 'ABBV.csv',
 'ABC.csv',
 'ABEV.csv',
 'ABG.csv',
 'ABM.csv',
 'ABR.csv']

In [ ]:
for path in filepaths:
  symbol = path.split('.')[0]
  df = pd.read_csv(path, index_col='datetime')
  entry_df = find_entries(df)
  trade_entries_dict[symbol] = entry_df
trade_entries_dict